In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 

In [2]:
from keras.models import model_from_json
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename) 
    img=None
    if (ds.BodyPartExamined=='CHEST' and ds.Modality=='DX' 
        and  ds.PatientPosition in ['AP', 'PA']):
        img = ds.pixel_array
    return img
    

# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    new_img=img.copy()
    new_img=(new_img-img_mean)/img_std
    new_img = np.resize(new_img,img_size)
    return new_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    #load model from json
    jsonfile=open(model_path,'r')
    loaded_model_json=jsonfile.read()
    jsonfile.close()
    loaded_model = model_from_json(loaded_model_json)
    #load weights now
    loaded_model.load_weights(weight_path)
    loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
    return loaded_model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(my_model, img, thresh):
    result = my_model.predict(img)  
    predict=result[0]
    prediction='No pneumonia'
    print('Result Prediction value ' + str(predict))
    if(predict>thresh):
        prediction='Pneumonia'
    return prediction 

In [3]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = 'my_model.json'#path to saved model
weight_path ='xray_class_model.best.hdf5'#path to saved best weights

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16


my_model = load_model(model_path, weight_path) #loads model
thresh = 0.372928 #loads the threshold determined after training the model. 


# use the .dcm files to test your prediction
for i in test_dicoms:

    img = np.array([])
    img = check_dicom(i)
    if img is None:
        print("Invalid Dicom record for Pneumonia...Rejected the record")
        continue
    img_mean = np.mean(img)# loads the mean image value they used during training preprocessing
    img_std =np.std(img) # loads the std dev image value they used during training preprocessing
       
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

Load file test1.dcm ...


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential/model/block1_conv1/Relu (defined at <ipython-input-2-e94f1099b84b>:37) ]] [Op:__inference_predict_function_816]

Function call stack:
predict_function
